# 92_ML 실습01 - 영화추천3

In [1]:
import pandas as pd
import numpy as np
movies = pd.read_csv('/Users/newrandom/Documents/GitHub/ML_study/Zerobase_part09_머신러닝/실습/Datasets/tmdb_5000_movies.csv')
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
                    # id : id
                        # title : 영화제목
                                # genres : 장르
                                            # vote_average : 평균 평점
                                                        # vote_count : 투표 수
                                                                        # popularity : 인기
                                                                                    # keywords : 키워드
                                                                                                # overview : 영화개요
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
                                                                            # x : movies_df['genres']
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:2]

# genres의 각 단어들을 하나의 문장(띄어쓰기로 구분) 으로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df.head()

## 문자열로 변환된 genres를 CountVectorize 수행
from sklearn.feature_extraction.text import CountVectorizer
                                            # 유사도를 만들어 보고자 함.

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
                                        # ngram_range : 보다 더 풍부하게 데이터를 보게하는 방법 중 하나
                                        # The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. 
                                        # All values of n such such that min_n <= n <= max_n will be used. 
                                        # For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. 
                                        # Only applies if analyzer is not callable.

genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

# genre_sim 객체에서 높은 값 순으로 정렬할 수 있다.
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
                                # numpy의 argsort()는 오름차순, 내림차순을 할 수 없다.
                                # 대신 [:, ::-1]을 하면 된다.


# 추천 영화를 DataFrame으로 반환하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
                                        # vector로 바꾸는 라인

    return df.iloc[similar_indexes]

# 실행
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

<ipython-input-1-8b30ff998d87>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(literal_eval)
<ipython-input-1-8b30ff998d87>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
<ipython-input-1-8b30ff998d87>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [2]:
# 평점과 평점을 매긴 횟수를 보면 문제 데이터가 보인다.
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

## 3519 의 경우 : vote_count가 1표인데, vote_average가 10 점임을 확인할 수도 있다.

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


![](/Users/newrandom/Documents/GitHub/ML_study/Zerobase_part09_머신러닝/실습/lecture_image/92-1.png)

In [3]:
# 영화 전체 평균평점과 최소 투표 횟수를 60% 지점으로 지정
C = movies_df['vote_average'].mean()
                            # 전체평균
m = movies_df['vote_count'].quantile(0.6)
                            # quantile : 분위수 // 60% 지점만 사용 지정
print('C : ', round(C,3), 'm : ', round(m,3))
                # 소수 3째자리까지 나타내기.

C :  6.092 m :  370.2


In [4]:
# 가중치가 부여된 평점을 계산하기 위한 함수
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ( (v/(v+m)) * R) + ( (m/(m+v)) * C)

In [5]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head()

<ipython-input-5-c93d821c3d90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,6.838594
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime,6.284091
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller,7.541095
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction,6.098838


In [6]:
movies_df.tail()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
4798,9367,El Mariachi,"[Action, Crime, Thriller]",6.6,238,14.269792,"[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,Action Crime Thriller,6.290894
4799,72766,Newlyweds,"[Comedy, Romance]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...,Comedy Romance,6.089611
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",7.0,6,1.444476,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TV Movie,6.106650
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],When ambitious New York attorney Sam is sent t...,,6.084894
4802,25975,My Date with Drew,[Documentary],6.3,16,1.929883,"[obsession, camcorder, crush, dream girl]",Ever since the second grade when he first saw ...,Documentary,6.100782


In [7]:
movies_df[movies_df['vote_count']<10]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
463,161795,Déjà Vu,"[Romance, Drama]",8.0,1,0.605645,"[love, american, pin, stranger, ruby]",L.A. shop owner Dana and Englishman Sean meet ...,Romance Drama,6.097311
492,293644,Top Cat Begins,"[Comedy, Animation]",5.3,9,0.719996,[3d],Top Cat has arrived to charm his way into your...,Comedy Animation,6.073370
1023,7504,Earth,[Drama],6.6,9,1.246883,"[based on novel, war of independence, period d...",It's 1947 and the borderlines between India an...,Drama,6.104224
1039,113464,Inchon,"[Drama, History, War]",6.5,2,0.146783,[],A noisy and absurd re-telling of the great 195...,Drama History War,6.094363
1453,49478,Warriors of Virtue,"[Fantasy, Family, Action]",4.7,9,0.912395,"[american football, mythology, chinese food, k...","A young man, Ryan, suffering from a disability...",Fantasy Family Action,6.059130
...,...,...,...,...,...,...,...,...,...,...
4795,124606,Bang,[Drama],6.0,1,0.918116,"[gang, audition, police fake, homeless, actress]",A young woman in L.A. is having a bad day: she...,Drama,6.091923
4797,67238,Cavite,"[Foreign, Thriller]",7.5,2,0.022173,[],"Adam, a security guard, travels from Californi...",Foreign Thriller,6.099736
4799,72766,Newlyweds,"[Comedy, Romance]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...,Comedy Romance,6.089611
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",7.0,6,1.444476,"[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TV Movie,6.106650


In [8]:
# 전체 데이터에서 가중치가 부여된 평점 순으로 정렬한 결과
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [ ]:
# # 이전 유사영화 찾기 함수
# def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
#     title_movie = df[df['title'] == title_name]

#     title_index = title_movie.index.values
#     similar_indexes = sorted_ind[title_index, :(top_n)]

#     print(similar_indexes)
#     similar_indexes = similar_indexes.reshape(-1)
#                                         # vector로 바꾸는 라인

#     return df.iloc[similar_indexes]

In [9]:
# 유사영화를 찾는 함수를 변경
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
                                    # vector로 바꾸는 라인 까지 이전과 같음.

    # 추가된 라인
    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]
                                    # 추가된 객체

In [10]:
# 다시 대부와 유사한 영화 찾기
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
